In [2]:
import glob
import cv2
import shutil
import os

def split_video_to_frames(video_path, output_dir):
    try:
        cap = cv2.VideoCapture(video_path)
        frame_count = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame_path = os.path.join(output_dir, f"frame_{frame_count:04d}.jpg")
            cv2.imwrite(frame_path, frame)
            frame_count += 1
        cap.release()
        logging.info(f"Total frames extracted from {video_path}: {frame_count}")
        return frame_count
    except Exception as e:
        logging.error(f"Error in split_video_to_frames: {e}")
        return 0

# Function to perform blob detection
def detect_blobs(image, min_area=1000, max_area=10000, min_circularity=0.05, min_convexity=0.5, min_inertia_ratio=0.01):
    try:
        # Convert image to grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur
        blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)

        # Apply thresholding
        _, thresh_image = cv2.threshold(blurred_image, 60, 255, cv2.THRESH_BINARY_INV)

        # Set up the blob detector
        params = cv2.SimpleBlobDetector_Params()
        params.filterByArea = True
        params.minArea = min_area
        params.maxArea = max_area
        params.filterByCircularity = True
        params.minCircularity = min_circularity
        params.filterByConvexity = True
        params.minConvexity = min_convexity
        params.filterByInertia = True
        params.minInertiaRatio = min_inertia_ratio
        params.filterByColor = True
        params.blobColor = 255  # Detect white blobs

        detector = cv2.SimpleBlobDetector_create(params)

        # Detect blobs
        keypoints = detector.detect(thresh_image)
        return keypoints
    except Exception as e:
        logging.error(f"Error in detect_blobs: {e}")
        return []

images = glob.glob("Dataset")
print(f"found {len(images)} original images")
new_dir = "./set2-rights-labeled-whole/"
try:
    os.mkdir(new_dir)
except FileExistsError:
    pass

for i, image_file in enumerate(images):
    img = cv2.imread(image_file)
    cv2.imshow("image", cv2.resize(img, (40*4, 40*4)))
    label = ""
    while label == "":
        ch = cv2.waitKey(-1)
        if ch == 27: # esc
            quit()
        if ch == ord('a'):
            label = "L"
        elif ch == ord('d'):
            label = "R"
        elif ch == ord('s'):
            label = "N"

    shutil.copyfile(image_file, f"{new_dir}{label}{str(i)}.png")

found 1 original images


error: OpenCV(4.10.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
